In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

# 전역 SparkContext 정리
SparkContext.setSystemProperty('spark.driver.host', '127.0.0.1')
SparkContext.setSystemProperty('spark.driver.bindAddress', '127.0.0.1')

# 모든 활성 context 중지
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# 새로 생성
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/16 18:13:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [3]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [5]:
import pandas as pd

In [6]:
df_pandas = pd.read_csv('head.csv')

In [7]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [8]:
from pyspark.sql import types

In [9]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [10]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [11]:
df = df.repartition(24)

In [12]:
df.write.parquet('fhvhv/2021/01/')

26/02/16 18:15:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/02/16 18:15:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/02/16 18:15:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [13]:
df = spark.read.parquet('fhvhv/2021/01/')

In [14]:
from pyspark.sql import functions as F

In [15]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [16]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [17]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

[Stage 5:>                                                          (0 + 1) / 1]

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-03|  2021-01-03|         132|          91|
|  e/9ce| 2021-01-01|  2021-01-01|         256|          65|
|  e/9ce| 2021-01-03|  2021-01-03|         191|         191|
|  e/9ce| 2021-01-03|  2021-01-03|         195|         265|
|  e/9ce| 2021-01-03|  2021-01-03|         119|         241|
|  s/acd| 2021-01-01|  2021-01-01|         148|          61|
|  e/9ce| 2021-01-01|  2021-01-01|         119|           1|
|  e/b32| 2021-01-01|  2021-01-01|         164|          68|
|  e/b38| 2021-01-01|  2021-01-01|         220|         240|
|  s/b13| 2021-01-03|  2021-01-03|          42|         254|
|  e/b3e| 2021-01-04|  2021-01-04|          23|          23|
|  e/b35| 2021-01-03|  2021-01-03|         251|         261|
|  e/95b| 2021-01-05|  2021-01-05|          35|          76|
|  e/b47| 2021-01-04|  2

In [18]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]